In [1]:
from flask import Flask, render_template, url_for, request, flash, redirect, jsonify, session
from flask_sqlalchemy import SQLAlchemy
import json
from datetime import datetime

# Global and Environment Variables
app = Flask(__name__)
app.secret_key = 'KEY'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
db = SQLAlchemy(app)

# The following line ensures that the above environment variables are available in the
# application context as needed.
# https://stackoverflow.com/questions/31444036/runtimeerror-working-outside-of-application-context
app.app_context().push()

# Database Models
class BlogPost(db.Model):
    """
    Represents a single blog post in the database.
    """
    id = db.Column(db.Integer, primary_key = True)
    title = db.Column(db.String(), nullable = False)
    content = db.Column(db.String(200), nullable = False)
    date_uploaded = db.Column(db.DateTime, default = datetime.utcnow)
    
    def __repr__(self):
        return '<Task %r>' % self.id
    
    @property
    def serialize(self):
        """
        Return the blog post as a dictionary. 
        """
        return {
            'id': self.id,
            'title': self.title,
            'content': self.content,
            'date_uploaded': self.date_uploaded,
        }
    
class Admin(db.Model):
    """
    Store username and password data for the admin user.
    """
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(120),nullable=False, unique = True)
    password = db.Column(db.String(120),nullable=False)

class Visits(db.Model):
    """
    Represents a single user interaction in the database.
    """
    id = db.Column(db.Integer, primary_key=True)
    page_id = db.Column(db.String(120), nullable = False)
    post_id = db.Column(db.Integer, nullable= True)
    time_stamp = db.Column(db.DateTime, default = datetime.utcnow)

# Utility Functions
def user_visits(page_id, post_id=None):
    visit = Visits()
    visit.page_id = page_id
    if post_id:
        visit.post_id = post_id
    db.session.add(visit)
    db.session.commit()


def create_admin_user():
    """
    Create an admin user with the default credentials.
    """
    default_admin=Admin(username = 'admin',password = 'password')
    db.session.add(default_admin)
    db.session.commit()

def add_post(post_title,post_content,post_datetime = None):
    """
    Add a blog post to the database.
    """
    new_blog=BlogPost(title=post_title,content=post_content,date_uploaded = datetime.fromisoformat(post_datetime))
    db.session.add(new_blog)
    db.session.commit()

def delete_post(blog_id):
    """
    Delete a post from the database
    """
    post = BlogPost.query.get_or_404(blog_id)
    db.session.delete(post)
    db.session.commit()

def edit_post(blog_id, new_title, new_content):
    """
    Update the title and content of a blog post in the database.
    """
    post = BlogPost.query.get_or_404(blog_id)
    post.title = new_title
    post.content = new_content
    db.session.commit()

def retrieve_blogs():
    """
    Return an iterable list of all blog posts in the database, beginning with the most recent post.
    """
    blogs = BlogPost.query.all()
    serialized_data = []
    for post in blogs:
        serialized_data.append(post.serialize)
    return serialized_data[::-1]

def retr_single_blog(blog_id):
    """
    Return a single serialized blog post. 
    """
    blog = BlogPost.query.get_or_404(blog_id)
    return blog.serialize

def initialize_database():
    """
    If a database does not exist, create a new database and populate it with default data.
    """
    try:
       admin = Admin.query.filter_by(username='admin').first() 
       print("Database found. Starting server with existing database.")
    except:
        print("No database found. Creating default database now.")
        with app.app_context():
            db.create_all()
        print("Database created. Populating with default data.")

        create_admin_user()
        print("Admin user created successfully.")

        with open('sampleposts.json') as sample_posts:
            default_posts = json.loads(sample_posts.read())['blog_posts']

        for post in default_posts:
            add_post(post['title'],post['content'],post['date_uploaded'])
        print("Default posts added successfully.")


# Routing
@app.route('/' or '/home')
def home_page():
    """
    Render the Home page.
    """
    user_visits('home')
    return render_template('index.html')

@app.route('/articles')
def articles():
    """
    Render the Articles page.
    """
    user_visits('articles')
    posts = retrieve_blogs()
    return render_template('articles.html', posts_list = posts)

@app.route('/admin', methods=('GET','POST'))
def admin():
    """
    Render the Admin page and handle all associated requests.
    """
    if 'authenticated' not in session or not session['authenticated']:
        return redirect('/login')
    elif request.method == 'POST':
        if request.form['form_id'] == 'add_post':
            post_title = request.form['title']
            post_content = request.form['content']
            add_post(post_title, post_content)
            flash(f'Blog post "{post_title}" added successfully!')
            return redirect('/admin')
        
        elif request.form['form_id'] == 'delete_post':
            post_title = request.form['title']
            post_id = request.form['blog_id']
            delete_post(post_id)
            flash(f'Blog post "{post_title}" deleted successfully!')
            return redirect('/admin')
        
        elif request.form['form_id'] == 'edit_post':
            post_data = retr_single_blog(request.form['blog_id'])
            return render_template('edit.html', post = post_data)
        
        elif request.form['form_id'] == 'post_editor':
            post_id = request.form['blog_id']
            new_title = request.form['title']
            new_content = request.form['content']
            edit_post(post_id, new_title, new_content)
            flash(f'Post ID# {post_id}: "{new_title}" has been edited successfully!')
            return redirect('/admin')
        
        elif request.form['form_id'] == 'logout':
            session['authenticated'] = False
            return redirect('/')
    else:
        posts = retrieve_blogs()
        return render_template('admin.html', posts_list = posts)

@app.route('/login', methods=['GET', 'POST'])
def login():
    """
    Render the login screen for the admin page and handle authentication.
    """
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        admin = Admin.query.filter_by(username=username).first()

        if admin and admin.password == password:
            session['authenticated'] = True
            return redirect('/admin')
        else:
            flash("Error: Username or password incorrect")
            return render_template('login.html')
    else:
        return render_template('login.html')
    
@app.route('/post/<int:post_id>', methods =['GET'])
def post(post_id=None):
    """
    Render the full text of a specific post for the user. 
    """
    selected_post = retr_single_blog(post_id)
    user_visits('post', post_id)
    return render_template('post.html',post = selected_post)

@app.route('/contact', methods = ['GET','POST'])
def contact():
    """
    Render the Contact page and handle form submissions. 
    """
    if request.method == 'GET':
        return render_template('contact.html')
    elif request.method == 'POST':
        flash("For submitted successfully. Thank you!")
        return render_template('contact.html')

# Error Code Handling
@app.errorhandler(404)
def not_found(e):
    """
    Execute this function if a request cannot be resolved. 
    """
    return render_template('404.html'), 404

def main():
    initialize_database()    
    app.run()

if __name__ == "__main__":
    main()

Database found. Starting server with existing database.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 20:10:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:10:43] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 20:10:45] "GET /articles HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 20:10:45] "GET /static/style.css HTTP/1.1" 304 -
